In [3]:
import sys
sys.path.append('../')

from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import os
import re
import time
import pickle
import requests
import html2text
from tqdm import tqdm
from pathlib import Path
from datetime import datetime
from langchain.docstore.document import Document

from config import get_logger

logger = get_logger(__name__)


# Settings for stackoverflow requests
key = os.getenv('SO_KEY')
tag = 'chainlink'
pagesize = 99
from_date = "1609459200"
to_date = "1685577600"

#https://api.stackexchange.com/docs/questions
def get_questions(tag, page, pagesize, from_date, to_date, key, access_token):
    # url = f'https://api.stackexchange.com/2.3/questions?order=desc&sort=creation&tagged={tag}&site={site}&pagesize={pagesize}&key={key}&access_token={access_token}&filter=withbody'
    url = f"https://api.stackexchange.com/2.3/questions?page={page}&pagesize={pagesize}&tagged={tag}&fromdate={from_date}&todate={to_date}&key={key}&access_token={access_token}&order=desc&sort=activity&site=stackoverflow&filter=withbody"
    response = requests.get(url)
    return response.json()['items']

def get_answers(question_id, key, access_token):
    url = f'https://api.stackexchange.com/2.3/questions/{question_id}/answers?order=desc&sort=activity&site=stackoverflow&key={key}&access_token={access_token}&filter=withbody'
    response = requests.get(url)
    return response.json()['items']

def get_all_questions(access_token):
    all_questions = None
    for i in range(1, 5):
        page = i
        if all_questions is None:
            all_questions = get_questions(
                tag=tag, pagesize=pagesize, key=key, 
                access_token=access_token, from_date=from_date, 
                to_date=to_date, page=page
            )
        else:
            all_questions.extend(get_questions(
                tag=tag, pagesize=pagesize, key=key, 
                access_token=access_token, from_date=from_date, 
                to_date=to_date, page=page
                )
            )
        time.sleep(3)

    all_questions = sorted(all_questions, key=lambda x: x['creation_date'], reverse=True)

    # Only consider questions from user with reputation > 50. Some questions don't have reputation field. They should be removed
    all_questions = [question for question in all_questions if 'reputation' in question['owner'].keys() and question['owner']['reputation'] > 50]

    h = html2text.HTML2Text()
    h.ignore_links = False
    h.ignore_images = True
    h.ignore_emphasis = False
    h.ignore_tables = False
    questions_and_answers = {}

    for question in tqdm(all_questions, total=len(all_questions)):
        answers = get_answers(question['question_id'], key, access_token)
        if not answers:  # Skip questions with no answers
            continue
        
        # Only keep accepted answer
        answers = [answer for answer in answers if answer['is_accepted'] == True]

        # Only keep answer from user with reputation > 50; some answers don't have reputation field. they should be removed
        answers = [answer for answer in answers if 'reputation' in answer['owner'].keys() and answer['owner']['reputation'] > 50]

        question_body = re.sub(r'!\[.*?\]\(.*?\)', '', h.handle(question['body']))
        question_str = "Question: (Asked on: " + datetime.utcfromtimestamp(question['creation_date']).strftime('%Y-%m-%d %H:%M:%S') + ")\n" + question['title'] + "\n"
        question_url = "URL: " + question['link'] + "\n\n"
        question_body_str = "Question Body:\n" + question_body + "\n\n"
        answers_str = "Answers:\n" + "\n---\n".join(["(Answered on: " + datetime.utcfromtimestamp(answer['creation_date']).strftime('%Y-%m-%d %H:%M:%S') + ")\n" + re.sub(r'!\[.*?\]\(.*?\)', '', h.handle(answer['body'])) for answer in answers]) + "\n\n"
        text = question_str + question_url + question_body_str + answers_str

        questions_and_answers[question['question_id']] = Document(page_content=text, metadata={"source":question['link'], "type":"stackoverflow"})

    logger.info("Scrapped stackoverflow")

    return questions_and_answers


def scrap_stackoverflow(access_token):
    logger.info(f"Key: {key}")
    logger.info(f"{access_token}")

    so_docs = get_all_questions(access_token)

    # Make sure data directory exists
    Path("./data").mkdir(parents=True, exist_ok=True)

    # Convert to list
    so_docs = list(so_docs.values())

    # Log first document
    logger.info(f"First document SO: {so_docs[0]}")

    # Save to disk
    with open("./data/stackoverflow_documents.pkl", "wb") as f:
        pickle.dump(so_docs, f)

    logger.info("Scrapped stackoverflow")

    return so_docs

In [5]:
code = "vmafUYae6trV9Bhsib4XAA))"

In [6]:
# https://stackoverflow.com/oauth?client_id=26442&scope=read_inbox&redirect_uri=https://stackapps.com
# curl -d "client_id=26442&client_secret=JtQuve9sdjhmgAdyOY3nKg((&code=vmafUYae6trV9Bhsib4XAA))&redirect_uri=https://stackapps.com" -X POST https://stackoverflow.com/oauth/access_token

In [7]:
access_token = "RqtrFkJZxBsVETNbooIb(A))"

In [8]:
qanda = get_all_questions(access_token)

100%|██████████| 154/154 [02:58<00:00,  1.16s/it]
2023-08-31 11:15:49,569 - __main__ - INFO - Scrapped stackoverflow
2023-08-31 11:15:49,569 - __main__ - INFO - Scrapped stackoverflow
INFO:__main__:Scrapped stackoverflow


In [9]:
so_docs = list(qanda.values())

In [11]:
with open("../data/stackoverflow_documents.pkl", "wb") as f:
    pickle.dump(so_docs, f)

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

In [5]:
# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Set up the webdriver
# s=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(options=chrome_options)

WebDriverException: Message: unknown error: Chrome failed to start: crashed.
  (unknown error: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /usr/bin/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x0040006a7783 <unknown>
#1 0x0040003d3917 <unknown>
#2 0x0040003fe978 <unknown>
#3 0x0040003fb9c9 <unknown>
#4 0x00400043ba13 <unknown>
#5 0x00400043b15f <unknown>
#6 0x004000432bd3 <unknown>
#7 0x004000406a7b <unknown>
#8 0x00400040781e <unknown>
#9 0x004000668e98 <unknown>
#10 0x00400066cd67 <unknown>
#11 0x00400067732c <unknown>
#12 0x00400066d993 <unknown>
#13 0x00400063c827 <unknown>
#14 0x0040006918a8 <unknown>
#15 0x004000691a36 <unknown>
#16 0x0040006a05f3 <unknown>
#17 0x004002552609 start_thread


In [1]:
import requests

In [14]:
res = requests.get("http://localhost:8000/oauth")

In [12]:
res.content

b'<!DOCTYPE html>\r\n\r\n\r\n    <html class="html__responsive html__unpinned-leftnav" lang="en">\r\n\r\n    <head>\r\n\r\n        <title>Log In - Stack Overflow</title>\r\n        <link rel="shortcut icon" href="https://cdn.sstatic.net/Sites/stackoverflow/Img/favicon.ico?v=ec617d715196">\r\n        <link rel="apple-touch-icon" href="https://cdn.sstatic.net/Sites/stackoverflow/Img/apple-touch-icon.png?v=c78bd457575a">\r\n        <link rel="image_src" href="https://cdn.sstatic.net/Sites/stackoverflow/Img/apple-touch-icon.png?v=c78bd457575a"> \r\n        <link rel="search" type="application/opensearchdescription+xml" title="Stack Overflow" href="/opensearch.xml">\r\n    <meta name="viewport" content="width=device-width, height=device-height, initial-scale=1.0, minimum-scale=1.0">\r\n        <meta property="og:type" content= "website" />\r\n        <meta property="og:url" content="https://stackoverflow.com/users/login?returnurl=https%3A%2F%2Fstackoverflow.com%2Foauth%3Fclient_id%3D26442%2

In [1]:
import os
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
load_dotenv()

# Configuration
CLIENT_ID=os.getenv("CLIENT_ID")
CLIENT_SECRET=os.getenv("CLIENT_SECRET")
LOGIN_URL = "https://stackoverflow.com/users/login"
REDIRECT_URI = "http://localhost"
OAUTH_URL = f"https://stackoverflow.com/oauth?client_id={CLIENT_ID}&scope=read_inbox&redirect_uri={REDIRECT_URI}"

session = requests.Session()

In [2]:
# 1. Log in
login_data = {
    "email": os.getenv("SO_LOGIN_EMAIL"),
    "password": os.getenv("SO_LOGIN_PASSWORD"),
    # You might need to fetch and provide any CSRF token or similar security token
    # This is just a basic example, and the exact form fields might differ
}
response = session.post(LOGIN_URL, data=login_data)
print(f"Login response: {response.status_code}")



Login response: 200
Authorize response: 200
Authorize response URL: https://stackoverflow.com/users/login?returnurl=https%3a%2f%2fstackoverflow.com%2foauth%3fclient_id%3d27148%26scope%3dread_inbox%26redirect_uri%3dhttp%3a%2f%2flocalhost


In [8]:
# 2. Authorize the app
response = session.get(OAUTH_URL, allow_redirects=False)
print(f"Authorize response: {response.status_code}")
print(f"Authorize response URL: {response.url}")

Authorize response: 302
Authorize response URL: https://stackoverflow.com/oauth?client_id=27148&scope=read_inbox&redirect_uri=http://localhost


In [10]:
response.headers

{'Connection': 'keep-alive', 'cache-control': 'private', 'location': '/users/login?returnurl=https%3a%2f%2fstackoverflow.com%2foauth%3fclient_id%3d27148%26scope%3dread_inbox%26redirect_uri%3dhttp%3a%2f%2flocalhost', 'strict-transport-security': 'max-age=15552000', 'x-frame-options': 'SAMEORIGIN', 'x-request-guid': '06cba4b3-cc17-4d83-a79f-6a753d9b7be9', 'set-cookie': 'prov=0e8d7f3c-0821-748c-a2ff-84c4387d66f4; expires=Sat, 07 Sep 2024 09:53:44 GMT; domain=.stackoverflow.com; path=/; secure; samesite=none; httponly', 'feature-policy': "microphone 'none'; speaker 'none'", 'content-security-policy': "upgrade-insecure-requests; frame-ancestors 'self' https://stackexchange.com", 'Accept-Ranges': 'bytes', 'Date': 'Thu, 07 Sep 2023 09:53:44 GMT', 'Via': '1.1 varnish', 'X-Served-By': 'cache-qpg1272-QPG', 'X-Cache': 'MISS', 'X-Cache-Hits': '0', 'X-Timer': 'S1694080425.689701,VS0,VE247', 'Vary': 'Fastly-SSL', 'X-DNS-Prefetch-Control': 'off', 'transfer-encoding': 'chunked'}

In [4]:
# 1. Check the URL for the code parameter
if "code=" in response.url:
    code = response.url.split("code=")[1].split("&")[0]  # Assuming code is not the last parameter
    print(f"Extracted code: {code}")
else:
    # 2. Check if there's an authorization form on the page
    soup = BeautifulSoup(response.text, 'html.parser')
    form = soup.find('form', {'id': 'oauth-authorize-form'})  # Placeholder form ID, inspect the actual page
    
    if form:
        action_url = form['action']
        payload = {}
        for input_tag in form.find_all('input'):
            payload[input_tag['name']] = input_tag['value']
        
        # Submit the authorization form
        response = session.post(action_url, data=payload, allow_redirects=False)
        
        # Now, check the response or the redirected URL for the code
        if "code=" in response.url:
            code = response.url.split("code=")[1].split("&")[0]
            print(f"Extracted code: {code}")
        else:
            print("Code not found after application authorization.")
    else:
        print("Unable to find the authorization form. Please inspect the page content.")


Unable to find the authorization form. Please inspect the page content.


In [2]:
# Create a session to maintain cookies
session = requests.Session()

# Try to access OAuth URL
response = session.get(OAUTH_URL, allow_redirects=False)
if response.status_code == 302 and 'location' in response.headers and '/users/login' in response.headers['location']:
    # We're being redirected to login
    
    login_data = {
        "email": os.getenv("SO_LOGIN_EMAIL"),
        "password": os.getenv("SO_LOGIN_PASSWORD")
    }
    response = session.post(LOGIN_URL, data=login_data)
    
    # Check if login was successful by looking at the response content or URL. This might require additional inspection.

    # Try to access OAuth URL again
    response = session.get(OAUTH_URL, allow_redirects=False)

# From here, you should handle the possibility of an authorization page or directly look for the code in the redirect URL.

In [4]:
response.url

'https://stackoverflow.com/oauth?client_id=27148&scope=read_inbox&redirect_uri=http://localhost'

In [7]:
response.headers.get("Location").split("code=")[1].split("&")[0]

'2RaFzOHFe5bQG0Q(KZ8(rQ))'

In [23]:
import requests

In [33]:
res = requests.get("http://localhost:8000/get_access_token")

In [34]:
res.content

b'{"access_token":"pNcNuGzj4*OgipiAz(MWkw))"}'